##### https://www.analyticsvidhya.com/blog/2020/08/analysing-streaming-tweets-with-python-and-postgresql/#h2_9

this saves tweets to out.csv

In [1]:
# Twitter API authentication

import tweepy
import config
import time



In [2]:
class MyStream(tweepy.Stream):
    
    
    def __init__(self, time_limit=300):
        
        self.start_time = time.time()
        self.limit = time_limit
        
        consumer_key = config.consumer_key
        consumer_secret = config.consumer_secret
        access_token = config.access_token
        access_token_secret = config.access_token_secret
        
        
        
        
        
        
        super().__init__(consumer_key,consumer_secret,access_token,access_token_secret)
    
        #self.file=open("tweetvidhya.csv","w")
    
    
    def on_connect(self):
        print("Connected to Twitter API.")
    

    
    def on_status(self, status):
        

        
        with open("out.csv", "a", encoding='utf-8') as f:
            f.write("%s,%s,%s,%s\n" % (status.created_at,status.user.screen_name,status.text,status.retweeted))

        
        
        
        # Tweet ID
        tweet_id = status.id
        
        # User ID
        user_id = status.user.id
        # Username
        username = status.user.name
        
        
        # Tweet
        if status.truncated == True:
            tweet = status.extended_tweet['full_text']
            hashtags = status.extended_tweet['entities']['hashtags']
        else:
            tweet = status.text
            hashtags = status.entities['hashtags']
        
        # Read hastags
        hashtags = read_hashtags(hashtags)            
        
        # Retweet count
        retweet_count = status.retweet_count
        # Language
        lang = status.lang
        
        
        # If tweet is not a retweet and tweet is in English
        if not hasattr(status, "retweeted_status") and lang=="en":
            # Connect to database
            dbConnect(user_id, username, tweet_id, tweet, retweet_count, hashtags)
            #pass
        if (time.time() - self.start_time) > self.limit:
            print(time.time(), self.start_time, self.limit)
            return False
            
    def on_error(self, status_code):
        if status_code == 420:
            # Returning False in on_data disconnects the stream
            return False

In [3]:
 # Extract hashtags
def read_hashtags(tag_list):
    hashtags = []
    for tag in tag_list:
        hashtags.append(tag['text'])
    return hashtags

In [4]:
import psycopg2
import config


# Insert Tweet data into database
def dbConnect(user_id, user_name, tweet_id, tweet, retweet_count, hashtags):
    
    # Connection to database server
    conn = psycopg2.connect(host="localhost",database="TwitterDB",port=5432,user='postgres',password=config.db_password)
    
    cur = conn.cursor()

    # insert user information
    command = '''INSERT INTO TwitterUser (user_id, user_name) VALUES (%s,%s) ON CONFLICT
                 (User_Id) DO NOTHING;'''
    cur.execute(command,(user_id,user_name))

    # insert tweet information
    command = '''INSERT INTO TwitterTweet (tweet_id, user_id, tweet, retweet_count) VALUES (%s,%s,%s,%s);'''
    cur.execute(command,(tweet_id, user_id, tweet, retweet_count))
    
    # insert entity information
    for i in range(len(hashtags)):
        hashtag = hashtags[i]
        command = '''INSERT INTO TwitterEntity (tweet_id, hashtag) VALUES (%s,%s);'''
        cur.execute(command,(tweet_id, hashtag))
    
    # Commit changes
    conn.commit()
    
    # Disconnect
    cur.close()
    conn.close()

In [5]:
# Streaming tweets

#myStream = MyStream(api_key, api_secret_key, access_token, access_token_secret)
myStream = MyStream()
#myStream = tweepy.Stream()#auth=api.auth, listener=myStreamListener,
#                        tweet_mode="extended")

#a = MyStream(api_key, api_secret_key, access_token, access_token_secret)                      


In [6]:
myStream.filter(track=['#covid-19'], languages = ["en"], threaded=True)


<Thread(Tweepy Stream, started 12456)>

Connected to Twitter API.
1630529502.1968002 1630529201.7474835 300
1630529505.804349 1630529201.7474835 300
1630529509.3558311 1630529201.7474835 300
1630529541.3674638 1630529201.7474835 300
1630529550.5526867 1630529201.7474835 300
1630529587.4322 1630529201.7474835 300
1630529603.441754 1630529201.7474835 300
1630529613.9801388 1630529201.7474835 300
